# Prédiction d'un résultat

In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import metrics

### Importation du csv. Par défaut, le fichier que l'on récupère provient de github, raw indique que ce ne sont que les données brutes, sans les balises html.

__Si jamais vous ne possédez pas internet, commentez le premier read_csv, et décommentez le second, en supposant que vous avez cloné le projet git en local__

In [118]:
data_result_match = pd.read_csv("https://raw.githubusercontent.com/Phignis/predict-women-soccer-results/main/src/ressources/data/woman_international_cup_results.csv", sep=",")
# data_result_match = pd.read_csv("ressources/data/woman_international_cup_results.csv", sep=",")

In [119]:
print(data_result_match.shape)

(4296, 9)


In [120]:
# print(data_result_match)
data_result_match.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1969-11-01,Italy,France,1,0,Euro,Novara,Italy,False
1,1969-11-01,Denmark,England,4,3,Euro,Aosta,Italy,True
2,1969-11-02,England,France,2,0,Euro,Turin,Italy,True
3,1969-11-02,Italy,Denmark,3,1,Euro,Turin,Italy,False
4,1975-08-25,Thailand,Australia,3,2,AFC Championship,Hong Kong,Hong Kong,True


> A présent, mettons en forme les données, afin qu'elles soient exploitables par skLearn. Le but est de transformer toutes données ayant un intêrêt en colonne d'entier. Nous allons donc modifier la colonne home_team, away_team, tournament et le couple home_team/away_team/neutral

In [121]:
""" On ajoute d'abord une colonne Win pour savoir quelle équipe a gagné. 
 0 égalité
 1 victoire home
 2 victoire away """

for index in data_result_match.index :
    if(data_result_match.loc[index, 'home_score'] == data_result_match.loc[index, 'away_score']) :
      data_result_match.loc[index, 'Win'] = 0
    elif(data_result_match.loc[index, 'home_score'] > data_result_match.loc[index, 'away_score']) :
      data_result_match.loc[index, 'Win'] = 1
    else :
      data_result_match.loc[index, 'Win'] = 2

data_result_match['Win'] = data_result_match['Win'].astype(int)


In [122]:
""" On ajoute une colonne id_away_team et id_home_team pour connaitre les équipes ayant joué, sous forme numérique pour traitement"""
listePays = sorted(list(set(np.concatenate([data_result_match['home_team'], data_result_match['away_team']]))))
"""
listePays contient ici un set de toutes les home_teams et away_teams, de manière unique. On le stocke dans un set, pour garentir l'unicité de chaque équipe dans tout le match
"""

for index in data_result_match.index :
  data_result_match.loc[index, 'id_home_team']= listePays.index(data_result_match.loc[index, 'home_team'])
  data_result_match.loc[index, 'id_away_team']= listePays.index(data_result_match.loc[index, 'away_team'])

data_result_match['id_home_team'] = data_result_match['id_home_team'].astype(int) # pour avoir des entiers, par défaut c'étaient des floats
data_result_match['id_away_team'] = data_result_match['id_away_team'].astype(int) # pour avoir des entiers, par défaut c'étaient des floats

In [123]:
"""
Actuellement, les prédictions prennent entre autres id_home_team, id_away_team et neutral. Pourtant, en terme de prédiction, un France Italy sur terrain neutre et l'équivalent
d'un Italy France sur terrain neutre. Pourtant actuellement, nos modèles ne peuvent l'effectuer. Pour éviter cela, le but est de générer un id unique, en fonction de la situation.
Les deux situations citées précédemment possèderaient donc le même id, tandis que les mêmes équipes sur terrain non neutre deviendrait un id différent.
L'implémentation choisie est d'effectuer un dictionnaire, avec pour clé le tuple (id_team_1, id_team_2, terrain_neutre), et en value son id Unique, auto-incrémenté
"""

# on commence par récupéré toute la liste des équipes de notre dataset, pour pouvoir générer tout les tuples possibles, on l'a déjà calculé, c'est listePays
listeTuple = [0] * pow(len(listePays), 2)
i = 0
for tuple1 in listePays:
  for tuple2 in listePays:
    listeTuple[i] = (tuple1, tuple2)
    i += 1
# on a généré tout les tuples possibles, y compris les (i, i), les (i, j) et (j, i). On ne veut pas les (i, i), on va directement les supprimer
listeTuple = [tuple for tuple in listeTuple if tuple[0] != tuple[1]]

etat_match_id = dict()
i = 0
# il reste a les rentrer dans le dictionnaire
for tuple in listeTuple:
  etat_match_id[tuple + (True, )] = i
  i += 1
  etat_match_id[tuple + (False, )] = i
  i += 1

# il reste un soucis, on veut que (i, j, true) et (j, i, true) ait le même id, ce qui n'est pas le cas actuellement. Le but est donc de tout bien remettre en place
for current_tuple in etat_match_id:
  if(current_tuple[2] is True):
    # alors pour (i, j, true), il existe forcément (j, i, true)
    etat_match_id[(current_tuple[1], current_tuple[0], current_tuple[2])] = etat_match_id[current_tuple]


# notre dictionnaire est prêt, reste a générer la nouvelle colonne, nommé etat_match
for index in data_result_match.index :
  data_result_match.loc[index, 'etat_match'] = etat_match_id[(data_result_match.loc[index, 'home_team'], data_result_match.loc[index, 'away_team'], data_result_match.loc[index, 'neutral'])]
data_result_match['etat_match'] = data_result_match['etat_match'].astype(int)

In [124]:
""" On ajoute une colonne id_tournois pour connaitre les tournois, sous forme numérique pour traitement"""

liste_tournois = sorted(list(pd.unique(data_result_match['tournament'])))
for index in data_result_match.index :
  data_result_match.loc[index, 'id_tournois'] = liste_tournois.index(data_result_match.loc[index, 'tournament'])
data_result_match['id_tournois'] = data_result_match['id_tournois'].astype(int)

In [125]:
"""On affiche la DataFrame pour savoir à quoi elle ressemble"""

data_result_match.head(5)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Win,id_home_team,id_away_team,etat_match,id_tournois
0,1969-11-01,Italy,France,1,0,Euro,Novara,Italy,False,1,90,64,34509,13
1,1969-11-01,Denmark,England,4,3,Euro,Aosta,Italy,True,1,47,54,18060,13
2,1969-11-02,England,France,2,0,Euro,Turin,Italy,True,1,54,64,20754,13
3,1969-11-02,Italy,Denmark,3,1,Euro,Turin,Italy,False,1,90,47,34475,13
4,1975-08-25,Thailand,Australia,3,2,AFC Championship,Hong Kong,Hong Kong,True,1,170,10,4158,2


In [126]:
print(etat_match_id["Italy", 'France', False])
print(etat_match_id["Denmark", 'England', True])

34509
18060


In [127]:
matriceCorrelation = data_result_match.corr()
print(matriceCorrelation['Win'])

home_score     -0.237448
away_score      0.529250
neutral        -0.033951
Win             1.000000
id_home_team    0.004386
id_away_team    0.026386
etat_match      0.026048
id_tournois     0.027258
Name: Win, dtype: float64


In [128]:
""" Ici nous choisissons d'utiliser 75% du dataset pour entraîner le modèle et 25% pour effectuer nos tests."""
# x_train, x_test, y_train, y_test = train_test_split(data_result_match.loc[:, data_result_match.columns.difference(['date', 'Win', 'home_team', 'away_team', 'city', 'country', 'tournament', 'away_score', 'home_score'])], data_result_match['Win'], test_size = 0.25, random_state = 0)
x_train, x_test, y_train, y_test = train_test_split(data_result_match[['neutral', 'id_home_team', 'id_away_team', 'id_tournois']], data_result_match['Win'], test_size = 0.25, random_state = 0)



In [129]:
""" Nous entrainons le modèle en utilisant la méthode de classification avec l'arbre de décision"""

arbre_decision = DecisionTreeClassifier(random_state = 0, max_depth = 20)
clf = arbre_decision.fit(x_train, y_train)




In [130]:
""" Nous effectuons notre prédiction sur notre jeu de test en utilisant le modèle entrainé avec la méthode de classification avecc l'arbre de décision """

sPredict = clf.predict(x_test)

score = accuracy_score(y_test, sPredict)

print(score)
print(metrics.confusion_matrix(y_test, sPredict))

print(y_test.head(10))
# for x in sPredict :
  # print(x)

0.590316573556797
[[ 34  80  36]
 [ 64 427  83]
 [ 57 120 173]]
545     2
2232    2
3222    1
2194    1
2971    1
1746    2
3438    0
1811    1
220     1
3932    2
Name: Win, dtype: int64


In [131]:
""" Nous entrainons le modèle en utilisant la méthode de classification avec le plus proche voisin """

KNN = KNeighborsClassifier()
clf = KNN.fit(x_train, y_train)


In [132]:
""" Nous effectuons notre prédiction sur notre jeu de test en utilisant le modèle entrainé avec la méthode de classification avec le plus proche de voisin """

sPredict = clf.predict(x_test)

score = accuracy_score(y_test, sPredict);
print(score)

print(metrics.confusion_matrix(y_test, sPredict))


0.5642458100558659
[[ 18 103  29]
 [ 43 451  80]
 [ 30 183 137]]


In [133]:

""" Nous entrainons le modèle en utilisant la méthode de classification avec SVM """
clf = svm.SVC(gamma = 0.001)
clf.fit(x_train, y_train)


SVC(gamma=0.001)

In [134]:
""" Nous effectuons notre prédiction sur notre jeu de test en utilisant le modèle entrainé avec la méthode de classification avec SVM """

sPredict = clf.predict(x_test)

score = accuracy_score(y_test, sPredict);
print(score)
print(type(x_test))

0.5465549348230913
<class 'pandas.core.frame.DataFrame'>
